In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import pickle
import h5py
import spacy
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster

In [2]:
# Load Droid metadata
metadata_fp = "./all_droid_metadata.pkl"
df = pd.read_pickle(metadata_fp)

In [4]:
# Filter only successful demos with any language instruction
successful_df = df[df['success'] == True]
successful_with_inst_df = successful_df.loc[~((successful_df['language_instruction_1'] == '') & 
              (successful_df['language_instruction_2'] == '') & 
              (successful_df['language_instruction_3'] == ''))]
successful_with_inst_df.dropna(subset=['language_instruction_1', 'language_instruction_2', 'language_instruction_3'], how='all')

,Demo,filepath,recording_folder_path,success,language_instruction_1,language_instruction_2,language_instruction_3,colors_in_demo,num_actions,max_ee_velocity,...,lab,date,timestamp,scene_id,task,building,trajectory_length,wrist_cam_extrinsics,ext1_cam_extrinsics,ext2_cam_extrinsics
3,demo_100,gs://xembodiment_data/r2d2/r2d2-data-full/TRI/...,gs://xembodiment_data/r2d2/r2d2-data-full/TRI/...,True,Put the coffee capsule in the coffee maker,,,[],236,1.523148,...,TRI,2023-11-27,2023-11-27-15h-41m-41s,8.949827e+09,"Do any task, and then reset the scene.\n\nSugg...",TRI,237.0,"[0.2400873129864906, -0.04782164495006405, 0.4...","[0.0863697908298307, 0.6370271737027244, 0.421...","[0.23703966939939874, -0.5803646239333268, 0.6..."
4,demo_1000,gs://xembodiment_data/r2d2/r2d2-data-full/REAL...,gs://xembodiment_data/r2d2/r2d2-data-full/REAL...,True,Put the tomato inside the silver bowl,Pick the tomato and put it in the bowl,Put the toy tomato in the colourless bow,"[tomato, silver]",180,1.542357,...,REAL,2023-07-25,2023-07-25-14h-33m-12s,8.626796e+09,Move object into or out of container (ex: draw...,Glen's office,181.0,"[0.2865870255047073, -0.050823464446027894, 0....","[-0.0022482606920192078, -0.40645056851965927,...","[0.2806535855443205, 0.5100490134107646, 0.556..."
5,demo_10000,gs://xembodiment_data/r2d2/r2d2-data-full/RAIL...,gs://xembodiment_data/r2d2/r2d2-data-full/RAIL...,True,Remove the gray toy from the bowl and put in i...,Remove the grey doll from the bowl,Put the grey plush toy inside the sink.,"[gray, grey]",286,0.772116,...,RAIL,2023-10-22,2023-10-22-11h-27m-25s,1.560521e+08,Move object into or out of container (ex: draw...,RAIL,287.0,"[0.2305277513568585, -0.09859850976796172, 0.4...","[0.25415971284823435, -0.32439911544807526, 0....","[0.3569127895668578, 0.5353923486172769, 0.315..."
6,demo_10001,gs://xembodiment_data/r2d2/r2d2-data-full/TRI/...,gs://xembodiment_data/r2d2/r2d2-data-full/TRI/...,True,Pick the popcorn on the napkin using the fork ...,,,[],447,1.608746,...,TRI,2023-11-29,2023-11-29-10h-38m-10s,5.445648e+08,Move object into or out of container (ex: draw...,TRI,448.0,"[0.3198158524363394, 0.161796845437758, 0.3666...","[0.16701060169204435, 0.39995461738548554, 0.3...","[0.14098025509586265, -0.5490224802945189, 0.5..."
7,demo_10002,gs://xembodiment_data/r2d2/r2d2-data-full/CLVR...,gs://xembodiment_data/r2d2/r2d2-data-full/CLVR...,True,Put the cloth on the countertop shield,Pick up the cloth and put it on the glass,Hang the cloth on the glass barrier,[],154,1.613444,...,CLVR,2023-06-25,2023-06-25-18h-54m-45s,7.489911e+09,"Hang or unhang object (ex: towel on hook, clot...",KAIST,155.0,"[0.3013884475276124, -0.04683488909072194, 0.3...","[0.04093385111638796, 0.32573132392889403, 0.2...","[0.08229829614709473, -0.321123667067028, 0.23..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92219,demo_9987,gs://xembodiment_data/r2d2/r2d2-data-full/ILIA...,gs://xembodiment_data/r2d2/r2d2-data-full/ILIA...,True,Put the grey backpack on the black chair,Move the black bag from the brown couch to the...,Pick up the backpack and put it on the armchair,"[black, grey, brown]",342,1.707058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[]
92223,demo_9990,gs://xembodiment_data/r2d2/r2d2-data-full/IRIS...,gs://xembodiment_data/r2d2/r2d2-data-full/IRIS...,True,Move the white mug backwards.,Move the white mug to the right,Move the cup to the right,[white],176,0.950902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[]
92228,demo_9995,gs://xembodiment_data/r2d2/r2d2-data-full/CLVR...,gs://xembodiment_data/r2d2/r2d2-data-full/CLVR...,True,Hang the white cloth on the wooden board,Hang the cloth on wooden stand,Pick up the towel from the table and put it on...,[white],175,1.324791,...,CLVR,2023-06-25,2023-06-25-18h-03m-42s,7.489911e+09,"Hang or unhang object (ex: towel on hook, clot...",KAIST,176.0,"[0.32261260712040685, 0.10035459296743669, 0.5...","[0.04093385111638796, 0.32573132392889403, 0.2...","[0.082

In [5]:
# Combine the three language instructions into one
successful_with_inst_df.loc[:, 'combined_instructions'] = successful_with_inst_df[['language_instruction_1', 'language_instruction_2', 'language_instruction_3']].agg(' '.join, axis=1)

# Preprocess text
def preprocess_text(text):
    text = text.lower()  # Lowercase the text
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    return text

successful_with_inst_df.loc[:, 'combined_instructions'] = successful_with_inst_df['combined_instructions'].apply(preprocess_text)
successful_with_inst_df['combined_instructions'] = successful_with_inst_df['combined_instructions'].str.strip()
combined_inst_df = successful_with_inst_df[['Demo', 'combined_instructions', 'building', 'success']]
combined_inst_df

/var/folders/c1/5_fch2z17mz0h1_rqcpt6t380000gn/T/ipykernel_31842/2030233973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  successful_with_inst_df.loc[:, 'combined_instructions'] = successful_with_inst_df[['language_instruction_1', 'language_instruction_2', 'language_instruction_3']].agg(' '.join, axis=1)
/var/folders/c1/5_fch2z17mz0h1_rqcpt6t380000gn/T/ipykernel_31842/2030233973.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  successful_with_inst_df['combined_instructions'] = successful_with_inst

,Demo,combined_instructions,building,success
3,demo_100,put the coffee capsule in the coffee maker,TRI,True
4,demo_1000,put the tomato inside the silver bowl pick the...,Glen's office,True
5,demo_10000,remove the gray toy from the bowl and put in i...,RAIL,True
6,demo_10001,pick the popcorn on the napkin using the fork ...,TRI,True
7,demo_10002,put the cloth on the countertop shield pick up...,KAIST,True
...,...,...,...,...
92219,demo_9987,put the grey backpack on the black chair move ...,NaN,True
92223,demo_9990,move the white mug backwards move the white mu...,NaN,True
92228,demo_9995,hang the white cloth on the wooden board hang ...,KAIST,True
92229,demo_9996,close the bottom drawer close the drawer close...,Smith Hall Fetch Lab,True


In [13]:
# Extract objects from combined language instructions
nlp = spacy.load("en_core_web_sm")

def extract_object(sentence):
    doc = nlp(sentence)
    obj = [token.text for token in doc if token.dep_ in ['dobj', 'pobj']]
    # return " ".join(obj) if obj else ""
    return obj[0] if obj else ""

clustered_dict = defaultdict(list)

for demo_id, sentence in zip(combined_inst_df['Demo'], combined_inst_df['combined_instructions']):
    obj = extract_object(sentence)
    if obj in clustered_dict:
        clustered_dict[obj].append((demo_id, sentence))
    else:
        clustered_dict[obj] = [(demo_id, sentence)]

In [27]:
clustered_dict.keys()

dict_keys(['marker', 'lid', 'object', 'towel', 'cup', 'pen', 'bottle', 'drawer', 'cloth', 'block', 'door', 'spoon', 'toy', 'bowl', 'switch', 'mug', 'contents', 'button', 'objects', 'box', 'band', 'pot', 'tape', 'faucet', 'plate', 'cube', 'cable', 'can', 'screwdriver', 'packet', 'tap', 'cups', 'fork', 'bottles', 'book', 'shirt', 'glass', 'blocks', 'bag', 'stick', 'lever', 'ball', '', 'curtain', 'item', 'napkin', 'table', 'rope', 'paper', 'bin', 'spatula', 'nozzle', 'clothes', 'some', 'tissue', 'cap', 'plushie', 'container', 'handle', 'thing', 'remote', 'knife', 'top', 'pan', 'basket', 'ring', 'jar', 'bar', 'turn', 'things', 'pack', 'light', 'elastic', 'sponge', 'sharpie', 'brush', 'kettle', 'straw', 'clothing', 'cabinet', 'one', 'piece', 'ropes', 'pepper', 'lamp', 'slice', 'maker', 'tshirt', 'hanger', 'mouse', 'tube', 'pillow', 'knob', 'sachet', 'doll', 'orange', 'shaker', 'spout', 'duck', 'toaster', 'pencil', 'tool', 'plush', 'plushy', 'press', 'microwave', 'cellar', 'plastic', 'toys',

In [26]:
clustered_dict = {k: v for k, v in sorted(clustered_dict.items(), key=lambda item: len(item[1]), reverse=True)}

In [32]:
print(f'NUMBER OF TOTAL DEMOS: {sum(len(val) for val in clustered_dict.values())}')
for k, v in clustered_dict.items():
    print(f'{k} - {len(v)}')

NUMBER OF TOTAL DEMOS: 49895
marker - 4680
lid - 2493
object - 2431
towel - 2067
cup - 2012
pen - 1808
bottle - 1666
drawer - 1649
cloth - 1441
block - 1320
door - 1171
spoon - 1071
toy - 896
bowl - 882
switch - 759
mug - 691
contents - 682
button - 573
objects - 519
box - 475
band - 434
pot - 431
tape - 384
faucet - 352
plate - 352
cube - 331
cable - 310
can - 310
screwdriver - 307
packet - 291
tap - 279
cups - 269
fork - 259
bottles - 244
book - 240
shirt - 232
glass - 228
blocks - 217
bag - 208
stick - 204
lever - 202
ball - 197
 - 188
curtain - 180
item - 178
napkin - 178
table - 176
rope - 172
paper - 159
bin - 156
spatula - 155
nozzle - 150
clothes - 147
some - 146
tissue - 143
cap - 142
plushie - 141
container - 139
handle - 138
thing - 137
remote - 135
knife - 134
top - 134
pan - 133
basket - 131
ring - 131
jar - 126
bar - 125
turn - 121
things - 119
pack - 119
light - 108
elastic - 106
sponge - 102
sharpie - 101
brush - 100
kettle - 100
straw - 92
clothing - 91
cabinet - 90
on

In [15]:
# Compare similarity between extracted objects
keys = list(clusters_dict.keys())
sentences = []
for values in clusters_dict.values():
    for value in values:
        sentences.append(value[1])

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(keys)
similarity_matrix = cosine_similarity(tfidf_matrix)

HYPERPARAMETER_TAU = 9
linked = linkage(similarity_matrix, 'ward')
labels = fcluster(linked, t=HYPERPARAMETER_TAU, criterion='distance')

label_to_key = {label: key for label, key in zip(labels, keys)}

result_dict = {}
for label, demos in zip(labels, clusters_dict.values()):
    key = label_to_key[label]
    if key not in result_dict:
        result_dict[key] = []
    result_dict[key].extend(demos) 

result_dict = {k: v for k, v in sorted(result_dict.items(), key=lambda item: len(item[1]), reverse=True)}

In [16]:
FILE_PATH = "roughly_clustered_demos.pkl"
with open(FILE_PATH, "wb") as f:
    pickle.dump(result_dict, f)